In [80]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import os
import webbrowser
import re
from wordcloud import WordCloud, STOPWORDS
from collections import Counter
import pytz
from datetime import datetime

In [3]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\MUGUNTHAN.A.R\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [4]:
app_data=pd.read_csv("D:/Google-Play-Store-Analytics-main/Play Store Data.csv")
review_data=pd.read_csv("D:/Google-Play-Store-Analytics-main/User Reviews.csv")

In [5]:
app_data.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [6]:
review_data.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


In [7]:
merged_data=pd.merge(review_data, app_data[['App', 'Category', 'Rating']], on='App', how='inner')

In [8]:
app_data = app_data.dropna(subset=['Rating'])
for column in app_data.columns :
    app_data[column].fillna(app_data[column].mode()[0],inplace=True)
app_data.drop_duplicates(inplace=True)
app_data=app_data=app_data[app_data['Rating']<=5]
review_data.dropna(subset=['Translated_Review'],inplace=True)


C:\Users\MUGUNTHAN.A.R\AppData\Local\Temp\ipykernel_12476\2981955353.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  app_data[column].fillna(app_data[column].mode()[0],inplace=True)


In [9]:
app_data.dtypes

App                object
Category           object
Rating            float64
Reviews            object
Size               object
Installs           object
Type               object
Price              object
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object

In [10]:
paid_apps = app_data[app_data['Type'] == 'Paid']
paid_apps

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
234,TurboScan: scan documents and receipts in PDF,BUSINESS,4.7,11442,6.8M,"100,000+",Paid,$4.99,Everyone,Business,"March 25, 2018",1.5.2,4.0 and up
235,Tiny Scanner Pro: PDF Doc Scan,BUSINESS,4.8,10295,39M,"100,000+",Paid,$4.99,Everyone,Business,"April 11, 2017",3.4.6,3.0 and up
427,Puffin Browser Pro,COMMUNICATION,4.0,18247,Varies with device,"100,000+",Paid,$3.99,Everyone,Communication,"July 5, 2018",7.5.3.20547,4.1 and up
476,"Moco+ - Chat, Meet People",DATING,4.2,1545,Varies with device,"10,000+",Paid,$3.99,Mature 17+,Dating,"June 19, 2018",2.6.139,4.1 and up
477,Calculator,DATING,2.6,57,6.2M,"1,000+",Paid,$6.99,Everyone,Dating,"October 25, 2017",1.1.6,4.0 and up
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10690,FO Bixby,PERSONALIZATION,5.0,5,861k,100+,Paid,$0.99,Everyone,Personalization,"April 25, 2018",0.2,7.0 and up
10697,Mu.F.O.,GAME,5.0,2,16M,1+,Paid,$0.99,Everyone,Arcade,"March 3, 2017",1.0,2.3 and up
10760,Fast Tract Diet,HEALTH_AND_FITNESS,4.4,35,2.4M,"1,000+",Paid,$7.99,Everyone,Health & Fitness,"August 8, 2018",1.9.3,4.2 and up
10782,Trine 2: Complete Story,GAME,3.8,252,11M,"10,000+",Paid,$16.99,Teen,Action,"February 27, 2015",2.22,5.0 and up


In [11]:
paid_apps['Installs']=paid_apps['Installs'].str.replace(',','').str.replace('+','').astype(int)
paid_apps['Price']=paid_apps['Price'].str.replace('$','').astype(float)

C:\Users\MUGUNTHAN.A.R\AppData\Local\Temp\ipykernel_12476\878706825.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  paid_apps['Installs']=paid_apps['Installs'].str.replace(',','').str.replace('+','').astype(int)
C:\Users\MUGUNTHAN.A.R\AppData\Local\Temp\ipykernel_12476\878706825.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  paid_apps['Price']=paid_apps['Price'].str.replace('$','').astype(float)


In [12]:
paid_apps['Revenue']=paid_apps['Price']*paid_apps['Installs']

C:\Users\MUGUNTHAN.A.R\AppData\Local\Temp\ipykernel_12476\519522875.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  paid_apps['Revenue']=paid_apps['Price']*paid_apps['Installs']


In [13]:
paid_apps['Revenue']

234      499000.00
235      499000.00
427      399000.00
476       39900.00
477        6990.00
           ...    
10690        99.00
10697         0.99
10760      7990.00
10782    169900.00
10785     12000.00
Name: Revenue, Length: 613, dtype: float64

In [14]:
def convert_size(size):
    if pd.isnull(size) or 'Varies' in size:
        return None
    elif 'M' in size:
        return float(size.replace('M', ''))
    elif 'k' in size:
        return float(size.replace('k', '')) / 1024
    else:
        return None

In [15]:
app_data

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10834,FR Calculator,FAMILY,4.0,7,2.6M,500+,Free,0,Everyone,Education,"June 18, 2017",1.0.0,4.1 and up
10836,Sya9a Maroc - FR,FAMILY,4.5,38,53M,"5,000+",Free,0,Everyone,Education,"July 25, 2017",1.48,4.1 and up
10837,Fr. Mike Schmitz Audio Teachings,FAMILY,5.0,4,3.6M,100+,Free,0,Everyone,Education,"July 6, 2018",1.0,4.1 and up
10839,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE,4.5,114,Varies with device,"1,000+",Free,0,Mature 17+,Books & Reference,"January 19, 2015",Varies with device,Varies with device


In [16]:
app_data['Last Updated'] = pd.to_datetime(app_data['Last Updated'], errors='coerce')
app_data['Updated_Month'] = app_data['Last Updated'].dt.month

In [17]:
html_files_path="D:/"
if not os.path.exists(html_files_path):
    os.makedirs(html_files_path)


In [18]:
plot_containers=""

In [19]:
# Save each Plotly figure to an HTML file
def save_plot_as_html(fig, filename, insight):
    global plot_containers
    filepath = os.path.join(html_files_path, filename)
    html_content = pio.to_html(fig, full_html=False, include_plotlyjs='inline')
    # Append the plot and its insight to plot_containers
    plot_containers += f"""
    <div class="plot-container" id="{filename}" onclick="openPlot('{filename}')">
        <div class="plot">{html_content}</div>
        <div class="insights">{insight}</div>
    </div>
    """
    fig.write_html(filepath, full_html=False, include_plotlyjs='inline')

In [20]:
plot_width=400
plot_height=300
plot_bg_color='black'
text_color='white'
title_font={'size':16}
axis_font={'size':12}

In [21]:
#Figure 9
fig9=px.scatter(
    paid_apps,
    x='Revenue',
    y='Installs',
    color='Category',
    trendline='ols',
    title='Revenue from Paid Apps',
    color_discrete_sequence=px.colors.qualitative.Vivid,
    width=400,
    height=300
)
fig9.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white',
    title_font={'size':16},
    xaxis=dict(title_font={'size':12}),
    yaxis=dict(title_font={'size':12}),
    margin=dict(l=10,r=10,t=30,b=10)
)
#fig1.update_traces(marker=dict(pattern=dict(line=dict(color='white',width=1))))
save_plot_as_html(fig9,"ScatterPlot.html","The Scatter Plot shows a correlation between the Revenue from Paid Apps")

c:\Users\MUGUNTHAN.A.R\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\regression\linear_model.py:1783: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


In [22]:
#Sentiment Distribution
def rating_group(rating):
    if rating >= 4:
        return 'Positive'
    elif rating >=3:
        return 'Neutral'
    elif rating >=2:
        return 'Negative'
app_data['Rating_Group']=app_data['Rating'].apply(rating_group)

In [23]:
app_data.columns
sia = SentimentIntensityAnalyzer()

In [24]:
review_data['Sentiment_Score']=review_data['Translated_Review'].apply(lambda x: sia.polarity_scores(str(x))['compound'])

In [25]:
#Figure 4
sentiment_counts=review_data['Sentiment_Score'].value_counts(1000)
fig4=px.bar(
    x=sentiment_counts.index,
    y=sentiment_counts.values,
    labels={'x':'Sentiment Score','y':'Count'},
    title='Sentiment Distribution',
    barmode='stack',
    color=sentiment_counts.index,
    color_discrete_sequence=px.colors.sequential.RdBu,
    width=400,
    height=300
)
fig4.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white',
    title_font={'size':16},
    xaxis=dict(title_font={'size':12}),
    yaxis=dict(title_font={'size':12}),
    margin=dict(l=10,r=10,t=30,b=10)
)
#fig1.update_traces(marker=dict(pattern=dict(line=dict(color='white',width=1))))
save_plot_as_html(fig4,"Sentiment Graph 4.html","Sentiments in reviews show a mix of positive and negative feedback, with a slight lean towards positive sentiments")

In [26]:
filtered_data = merged_data[
    (merged_data['Category'] == 'HEALTH_AND_FITNESS') &
    (merged_data['Rating'] == 5.0) &
    (merged_data['Translated_Review'].notnull())
]

In [27]:
review_text = " ".join(filtered_data['Translated_Review'].astype(str).str.lower())

In [28]:
app_names = set(app_data[app_data['Category'] == 'HEALTH_AND_FITNESS']['App'].str.lower())

In [29]:
def remove_app_names(text, names):
    for name in names:
        name_clean = re.sub(r'[^a-zA-Z\s]', '', name)  # Remove special characters
        text = re.sub(r'\b' + re.escape(name_clean) + r'\b', '', text, flags=re.IGNORECASE)
    return text

In [30]:
cleaned_text = remove_app_names(review_text, app_names)

In [31]:
stopwords = set([
    'app', 'apps', 'use', 'used', 'using', 'get', 'got', 'one', 'really', 'great', 'awesome', 'nice',
    'good', 'like', 'helpful', 'help', 'easy', 'best', 'love', 'amazing', 'excellent'
])

In [32]:
words = re.findall(r'\b[a-z]{3,}\b', cleaned_text)
filtered_words = [word for word in words if word not in stopwords]
word_counts = Counter(filtered_words).most_common(100)

In [33]:
wordcloud_data = pd.DataFrame(word_counts, columns=['Word', 'Frequency'])
wordcloud_data['FontSize'] = wordcloud_data['Frequency'] * 3
wordcloud_data['X'] = pd.Series(range(len(wordcloud_data))) % 10
wordcloud_data['Y'] = pd.Series(range(len(wordcloud_data))) // 10


In [34]:
fig3 = px.scatter(
    wordcloud_data,
    x='X',
    y='Y',
    text='Word',
    size='FontSize',
    color='Frequency',
    title="Word Cloud (Plotly) from 5-Star Health & Fitness Reviews",
)

fig3.update_traces(
    mode='text',
    textfont=dict(size=wordcloud_data['FontSize'], family='Arial'),
    textposition='top center'
)

fig3.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white',
    title_font={'size':16},
    xaxis=dict(visible=False),
    yaxis=dict(visible=False),
    template='plotly_white',
    showlegend=False
)
save_plot_as_html(fig3,"Wordcolud.html","")


In [35]:
app_data['Size'] = app_data['Size'].apply(convert_size)

In [36]:
filtered_df = app_data[
    (app_data['Rating'] >= 4.0) &
    (app_data['Size'] >= 10) &
    (app_data['Updated_Month'] == 1)
]

In [37]:
grouped = filtered_df.groupby('Category').agg({
    'Installs': 'sum',
    'Rating': 'mean',
    'Reviews': lambda x: pd.to_numeric(x, errors='coerce').sum()
}).reset_index()

In [38]:
top10 = grouped.sort_values(by='Installs', ascending=False).head(10)


In [39]:
india_tz = pytz.timezone('Asia/Kolkata')
current_hour_ist = datetime.now(india_tz).hour

In [40]:
if 15 <= current_hour_ist < 17:
    # --- Plot using Plotly ---
    fig4 = go.Figure()

    # Bar for average rating
    fig4.add_trace(go.Bar(
        x=top10['Category'],
        y=top10['Rating'],
        name='Average Rating',
        marker_color='skyblue'
    ))

    # Bar for total review count
    fig4.add_trace(go.Bar(
        x=top10['Category'],
        y=top10['Reviews'],
        name='Total Reviews',
        marker_color='salmon'
    ))

    # Layout
    fig4.update_layout(
        barmode='group',
        title='Average Rating vs Total Reviews (Top 10 Categories by Installs)',
        xaxis_title='App Category',
        yaxis_title='Value',
        template='plotly_white',
        plot_bgcolor='black',
        paper_bgcolor='black',
        font_color='white',
        title_font={'size':16},
        xaxis=dict(title_font={'size':12}),
        yaxis=dict(title_font={'size':12}),
        margin=dict(l=10,r=10,t=30,b=10))

    save_plot_as_html(fig4,"GroupedBarChart","")
else:
    print("⏰ The graph is only displayed between 3 PM and 5 PM IST.")


⏰ The graph is only displayed between 3 PM and 5 PM IST.


In [52]:
app_data['Installs'] = app_data['Installs'].str.replace('[+,]', '', regex=True).astype(float)

In [53]:
df = app_data[~app_data['Category'].str.startswith(('A', 'C', 'G', 'S'))]

In [54]:
category_installs = df.groupby('Category')['Installs'].sum().reset_index()

In [55]:
top5_categories = category_installs.sort_values(by='Installs', ascending=False).head(5)['Category']
df_top5 = df[df['Category'].isin(top5_categories)]

In [ ]:
np.random.seed(42)
sample_countries = ['US', 'IN', 'BR', 'RU', 'DE', 'GB', 'CA', 'FR', 'JP', 'AU']
df_top5['Country'] = np.random.choice(sample_countries, size=len(df_top5))

C:\Users\MUGUNTHAN.A.R\AppData\Local\Temp\ipykernel_12476\316411455.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
grouped = df_top5.groupby(['Country', 'Category'])['Installs'].sum().reset_index()

In [58]:
highlighted = grouped[grouped['Installs'] > 1_000_000]

In [ ]:
india_tz = pytz.timezone('Asia/Kolkata')
current_hour_ist = datetime.now(india_tz).hour

In [71]:
if 18 <= current_hour_ist < 20:
    # Create Choropleth Map
    fig = px.choropleth(
        highlighted,
        color='Installs',
        hover_name='Category',
        locations='Country',
        locationmode='ISO-3',
        color_continuous_scale='RdBu',
        title='Global Installs by Category (Only >1 Million, Top 5 Categories)'
    )

    fig.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white',
    title_font={'size':16},
    xaxis=dict(title_font={'size':12}),
    yaxis=dict(title_font={'size':12}),
    )
    save_plot_as_html(fig,"Choropleth Graph.html","")

else:
    print("⏰ Choropleth map only available between 6 PM and 8 PM IST.")

In [61]:
app_data['Size'] = app_data['Size'].replace('M', '', regex=False).replace('k', '', regex=False)
app_data['Size'] = pd.to_numeric(app_data['Size'], errors='coerce')

In [62]:
app_data['Android Ver'] = pd.to_numeric(app_data['Android Ver'].str.extract(r'(\d+\.\d+)')[0], errors='coerce')

In [ ]:
app_data['App_Length'] = app_data['App'].str.len()


In [ ]:
filter_data = app_data[
    (app_data['Installs'] >= 10_000) &
    ((app_data['Type'] == 'Free') | (paid_apps['Revenue'] >= 10_000)) &
    (app_data['Android Ver'] > 4.0) &
    (app_data['Size'] >= 15) &
    (app_data['Content Rating'] == 'Everyone') &
    (app_data['App_Length'] <= 30)
]

In [ ]:
top3_categories = filter_data.groupby('Category')['Installs'].sum().sort_values(ascending=False).head(3).index
df_top3['Revenue'] = df_top3['Price'] * df_top3['Installs']
df_top3 = filter_data[filter_data['Category'].isin(top3_categories)]

In [ ]:
grouped = df_top3.groupby(['Category', 'Type']).agg({
    'Installs': 'mean',
    'Revenue': 'mean'
}).reset_index()


In [ ]:
ist = pytz.timezone('Asia/Kolkata')
current_hour = datetime.now(ist).hour

In [ ]:
if current_hour == 13:  # Between 1PM and 2PM IST
    # Dual-axis plot
    fig = go.Figure()

    fig.add_trace(go.Bar(
        x=grouped['Category'] + " (" + grouped['Type'] + ")",
        y=grouped['Installs'],
        name='Average Installs',
        yaxis='y1',
        marker_color='royalblue'
    ))

    fig.add_trace(go.Scatter(
        x=grouped['Category'] + " (" + grouped['Type'] + ")",
        y=grouped['Revenue'],
        name='Average Revenue',
        yaxis='y2',
        mode='lines+markers',
        marker_color='firebrick'
    ))

    fig.update_layout(
        title="Average Installs vs Revenue for Free vs Paid Apps (Top 3 Categories)",
        plot_bgcolor='black',
        paper_bgcolor='black',
        font_color='white',
        title_font={'size':16},
        xaxis_title="App Category (Type)",
        yaxis=dict(
            title='Average Installs',
            titlefont=dict(color='royalblue'),
            tickfont=dict(color='royalblue')
        ),
        yaxis2=dict(
            title='Average Revenue ($)',
            titlefont=dict(color='firebrick'),
            tickfont=dict(color='firebrick'),
            overlaying='y',
            side='right'
        ),
        template='plotly_white'
    )
    save_plot_as_html(fig,"Dualaxis.html","This graph shows the avearge installs vs free and paid apps")
else:
    print("⏰ This chart is only visible between 1 PM and 2 PM IST.")
    

⏰ This chart is only visible between 1 PM and 2 PM IST.


In [72]:
app_data['Month'] = app_data['Last Updated'].dt.to_period('M').astype(str)

In [73]:
data = app_data[
    ~app_data['App'].str.lower().str.startswith(('x', 'y', 'z')) &
    ~app_data['App'].str.contains('S', case=True, na=False)
]


In [74]:
data1 = data[data['Category'].str.startswith(('E', 'C', 'B'))]


In [75]:
data1['Reviews']=data1['Size'].astype(float)

C:\Users\MUGUNTHAN.A.R\AppData\Local\Temp\ipykernel_12476\2778889227.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [76]:
data1 = data1[data1['Reviews'] > 500]

In [77]:
category_translation = {
    'Beauty': 'सौंदर्य (Hindi)',
    'Business': 'வணிகம் (Tamil)',
    'Dating': 'Partnersuche (German)'
}
data1['Category'] = data1['Category'].replace(category_translation)

In [78]:
ist = pytz.timezone('Asia/Kolkata')
current_hour = datetime.now(ist).hour

In [81]:
if 18 <= current_hour <= 21:
    # Group by month and category
    grouped = app_data.groupby(['Month', 'Category'])['Installs'].sum().reset_index()
    grouped.sort_values(by='Month', inplace=True)

    fig = go.Figure()

    for cat in grouped['Category'].unique():
        cat_data = grouped[grouped['Category'] == cat].copy()
        cat_data['Prev'] = cat_data['Installs'].shift(1)
        cat_data['Growth'] = ((cat_data['Installs'] - cat_data['Prev']) / cat_data['Prev']) * 100
        cat_data['Highlight'] = cat_data['Growth'] > 20

        # Main Line
        fig.add_trace(go.Scatter(
            x=cat_data['Month'], y=cat_data['Installs'],
            mode='lines+markers', name=cat
        ))

        # Shading areas of >20% growth
        fig.add_trace(go.Scatter(
            x=cat_data['Month'],
            y=cat_data['Installs'].where(cat_data['Highlight'], None),
            fill='tozeroy',
            mode='none',
            name=f'{cat} (Growth > 20%)',
            fillcolor='rgba(255,0,0,0.2)',
            showlegend=False
        ))

    fig.update_layout(
        title="📈 Install Trend Over Time (Highlighted: >20% Monthly Growth)",
        plot_bgcolor='black',
        paper_bgcolor='black',
        font_color='white',
        title_font={'size':16},
        xaxis=dict(title_font={'size':12}),
        yaxis=dict(title_font={'size':12}),
        xaxis_title="Month",
        yaxis_title="Total Installs",
        template="plotly_white"
    )

    save_plot_as_html(fig,"ScatterPlot2.html","")

else:
    print("⏰ This graph is only visible between 6 PM and 9 PM IST.")


In [101]:
app_data['Installs'] = app_data['Installs'].astype(float)

In [102]:
app_data['Size'] = pd.to_numeric(app_data['Size'], errors='coerce')
app_data['Rating'] = pd.to_numeric(app_data['Rating'], errors='coerce')
app_data['Reviews'] = pd.to_numeric(app_data['Reviews'], errors='coerce')

In [103]:
app_data['App_Length'] = app_data['App'].str.len()

In [104]:
allowed_categories = ['Game', 'Beauty', 'Business', 'Comics', 'Communication', 'Dating', 'Entertainment', 'Social', 'Events']

In [105]:
review_data = review_data.dropna(subset=['Translated_Review', 'Sentiment_Subjectivity'])
avg_subjectivity = review_data.groupby('App')['Sentiment_Subjectivity'].mean().reset_index()



In [106]:
merged = pd.merge(app_data, avg_subjectivity, on='App', how='inner')

In [107]:
filtered = merged[
    (merged['Rating'] > 3.5) &
    (merged['Reviews'] > 500) &
    (merged['Installs'] > 50000) &
    (merged['Category'].isin(allowed_categories)) &
    (~merged['App'].str.contains('S')) &
    (merged['Sentiment_Subjectivity'] > 0.5)
].copy()

In [108]:
filtered['Translated_Category'] = filtered['Category'].replace(category_translation)
filtered['Translated_Category'] = filtered['Translated_Category'].fillna(filtered['Category'])


In [109]:
india_tz = pytz.timezone('Asia/Kolkata')
current_hour = datetime.now(india_tz).hour


In [111]:
if 17 <= current_hour < 19:  # Between 5 PM and 7 PM IST
    # Bubble chart using Plotly
    fig = px.scatter(
        filtered,
        x='Size_MB',
        y='Rating',
        size='Installs',
        color='Translated_Category',
        hover_name='App',
        title='📊 App Size vs Rating (Bubble Size = Installs)',
        size_max=60
    )

    # Highlight "Game" category in pink
    for i, d in enumerate(fig.data):
        if d.name == 'Game':
            fig.data[i].marker.color = 'deeppink'

    fig.update_layout(
        xaxis_title='App Size (MB)',
        yaxis_title='Average Rating',
        plot_bgcolor='black',
        paper_bgcolor='black',
        font_color='white',
        title_font={'size':16},
        xaxis=dict(title_font={'size':12}),
        yaxis=dict(title_font={'size':12}),
        template='plotly_white'
    )

    fig.show()

else:
    print("⏰ This bubble chart is only visible between 5 PM and 7 PM IST.")


⏰ This bubble chart is only visible between 5 PM and 7 PM IST.


In [112]:
dashboard_html= """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name=viewport" content="width=device-width,initial-scale-1.0">
    <title> Google Play Store Review Analytics</title>
    <style>
        body {{
            font-family: Arial, sans-serif;
            background-color: #333;
            color: #fff;
            margin: 0;
            padding: 0;
        }}
        .header {{
            display: flex;
            align-items: center;
            justify-content: center;
            padding: 20px;
            background-color: #444
        }}
        .header img {{
            margin: 0 10px;
            height: 50px;
        }}
        .container {{
            display: flex;
            flex-wrap: wrap;
            justify_content: center;
            padding: 20px;
        }}
        .plot-container {{
            border: 2px solid #555
            margin: 10px;
            padding: 10px;
            width: {plot_width}px;
            height: {plot_height}px;
            overflow: hidden;
            position: relative;
            cursor: pointer;
        }}
        .insights {{
            display: none;
            position: absolute;
            right: 10px;
            top: 10px;
            background-color: rgba(0,0,0,0.7);
            padding: 5px;
            border-radius: 5px;
            color: #fff;
        }}
        .plot-container: hover .insights {{
            display: block;
        }}
        </style>
        <script>
            function openPlot(filename) {{
                window.open(filename, '_blank');
                }}
        </script>
    </head>
    <body>
        <div class= "header">
            <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/4/4a/Logo_2013_Google.png/800px-Logo_2013_Google.png" alt="Google Logo">
            <h1>Google Play Store Reviews Analytics</h1>
            <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/7/78/Google_Play_Store_badge_EN.svg/1024px-Google_Play_Store_badge_EN.svg.png" alt="Google Play Store Logo">
        </div>
        <div class="container">
            {plots}
        </div>
    </body>
    </html>
    """


In [113]:
final_html=dashboard_html.format(plots=plot_containers,plot_width=plot_width,plot_height=plot_height)

In [114]:
dashboard_path=os.path.join(html_files_path,"web page.html")

In [115]:
with open(dashboard_path, "w", encoding="utf-8") as f:
    f.write(final_html)

In [116]:
webbrowser.open('file://'+os.path.realpath(dashboard_path))

True